In [1]:
options(Seurat.object.assay.version = 'v3') 

library(Seurat)
library(ggridges)
library(ggplot2)
library(Mixscale)
library(Seurat)
library(ggplot2)
library(patchwork)
library(scales)
library(dplyr)
library(reshape2)
library(cowplot)
library(harmony)
library(pROC)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Warning message:
“replacing previous import ‘ggplot2::vars’ by ‘glmGamPoi::vars’ when loading ‘Mixscale’”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots


Loading required package: Rcpp

Registered S3 method overwritten by 'pROC':
  method   from            
  plot.roc spatstat.explore

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




In [ ]:
#Reload complete seurat object
merged_counts_filtered_combined <- readRDS("/data/tracy_data/perturbseq_qc.rds")

In [2]:
# Reload TCDD and pbs seurat object
merged_counts_perturb1_tcdd_pbs <- readRDS("/data/tracy_data/perturbseq_qc_tcddafterharmony.rds")

In [ ]:
# Reload Calcification and DMEM seurat object
merged_counts_perturb1_cal_dmem <- readRDS("/data/tracy_data/perturbseq_qc_calcificationafterharmony.rds")

In [ ]:
UpdateSeuratObject(merged_counts_filtered_combined) 

In [ ]:
UpdateSeuratObject(merged_counts_perturb1_tcdd_pbs) 

In [ ]:
UpdateSeuratObject(merged_counts_perturb1_cal_dmem) 

In [ ]:
########################################################################################################################

In [ ]:
###load complete seurat object, followed by normalization and add guides info

In [ ]:
merged_counts_filtered_combined <- subset(merged_counts_filtered_combined, subset = nFeature_RNA > 1500 & 
                             nFeature_RNA < 10000 &  nCount_RNA > 2500 & percent.mt < 7.5)

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 12)
VlnPlot(merged_counts_filtered_combined, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, pt.size = 0.1)

In [ ]:
table(merged_counts_filtered_combined$group1)

In [ ]:
# Normalization
merged_counts_filtered_combined_no <- NormalizeData(merged_counts_filtered_combined, normalization.method = "LogNormalize", scale.factor = 10000)
# Feature selection
merged_counts_filtered_combined_no <- FindVariableFeatures(merged_counts_filtered_combined_no, selection.method = "vst", nfeatures = 2000)
# Scaling the data
all.genes <- rownames(merged_counts_filtered_combined_no)
merged_counts_filtered_combined_no <- ScaleData(merged_counts_filtered_combined_no, features = all.genes)
#These are now standard steps in the Seurat workflow for visualization and clustering
merged_counts_filtered_combined_no <- RunPCA(merged_counts_filtered_combined_no, verbose = FALSE)
merged_counts_filtered_combined_no <- RunUMAP(merged_counts_filtered_combined_no, dims = 1:50, verbose = FALSE)
merged_counts_filtered_combined_no <- FindNeighbors(merged_counts_filtered_combined_no, dims = 1:50, verbose = FALSE)
##resolution
merged_counts_filtered_combined_no <- FindClusters(merged_counts_filtered_combined_no, resolution = 0.5, verbose = FALSE)
DimPlot(merged_counts_filtered_combined_no, group.by = "group1")
FeaturePlot(merged_counts_filtered_combined_no, features = "nCount_RNA", min.cutoff = 1000)

In [ ]:
######add guides information##########

In [ ]:
perturb_counts <- read.csv('/VOSNE/tracy/ahr_perturbseq/outs/count/crispr_analysis/protospacer_calls_per_cell.csv', header = TRUE)

In [ ]:
##filter NA cells without guides information
row_ids <- perturb_counts$cell_barcode
merged_counts_filtered <- merged_counts_filtered_combined_no[,(colnames(merged_counts_filtered_combined_no) %in% row_ids)]
merged_counts_filtered[[]]
##Add guides information
merged_counts_perturb <- NormalizeData(
  object = merged_counts_filtered, 
  assay = "feature", 
  normalization.method = "CLR", 
  margin = 2)
merged_counts_perturb@meta.data$cell_barcode <- row.names(merged_counts_perturb@meta.data)
merged_counts_perturb@meta.data <- merged_counts_perturb@meta.data %>% left_join(y=perturb_counts, by = c('cell_barcode'))
merged_counts_perturb@meta.data$cell_barcode <- row.names(merged_counts_perturb@meta.data)
merged_counts_perturb@meta.data
#Add rowname vector back to object metadata
row_names <- rownames(x = merged_counts_filtered@meta.data)
rownames(merged_counts_perturb@meta.data) <- row_names
merged_counts_perturb[[]]
table(merged_counts_perturb$num_features)

In [ ]:
##another method to specify guides##
##subset only 1 guide##
selected_identname <- "num_features"
selected_ident <- c(1)
selected_rownumbers <- which(merged_counts_perturb[[]][,selected_identname] %in% selected_ident)
selected_cells <- rownames(merged_counts_perturb[[]][selected_rownumbers,])
merged_counts_perturb_1guide_object <- subset(merged_counts_perturb, cells = selected_cells) # subset from previously selected cells
##subset only 2 guides##
selected_identname <- "num_features"
selected_ident <- c(2)
selected_rownumbers <- which(merged_counts_perturb[[]][,selected_identname] %in% selected_ident)
selected_cells <- rownames(merged_counts_perturb[[]][selected_rownumbers,])
merged_counts_perturb_2guide_object <- subset(merged_counts_perturb, cells = selected_cells) # subset from previously selected cells
#Idents(merged_counts_perturb_2guide_object) <- "feature_call"
#twoguides <- table(merged_counts_perturb_2guide_object$feature_call)
#write.csv(twoguides, "/home/tracy/twoguides.csv")
#chatgpt
merged_counts_perturb_2guide_object@meta.data$guide3 <- grepl("TWIST1.*\\|TWIST1.*|NEG_CTRL.*\\|NEG_CTRL.*|TCF2.*\\|TCF2.* |ARNT.*\\|ARNT.*|ARNT.*\\|NEG_CTRL.*|HSP90AA1.*\\|HSP90AA1.*|HSP90AA1.*\\|NEG_CTRL.*|KLF4.*\\|KLF4.*|KLF4.*\\|NEG_CTRL.*|NFE2L2.*\\|NFE2L2.*|NFE2L2.*\\|NEG_CTRL.*|SMAD3.*\\|SMAD3.*|SMAD3.*\\|NEG_CTRL.*|SMARCA4.*\\|SMARCA4.*|SMARCA4.*\\|NEG_CTRL.*|SOX9.*\\|SOX9.*|SOX9.*\\|NEG_CTRL.*|TIPARP.*\\|TIPARP.*|TIPARP.*\\|NEG_CTRL.*|TWIST1.*\\|NEG_CTRL.*|TCF2.*\\|NEG_CTRL.*|TCF21.*\\|TCF21.*", merged_counts_perturb_2guide_object$feature_call)
##subset cells with guide3 is TRUE##
selected_identname <- c("guide3")
selected_ident <- c("TRUE")
selected_rownumbers <- which(merged_counts_perturb_2guide_object[[]][,selected_identname] %in% selected_ident)
selected_cells <- rownames(merged_counts_perturb_2guide_object[[]][selected_rownumbers,])
merged_counts_perturb_2guide_object <- subset(merged_counts_perturb_2guide_object, cells = selected_cells) # subset from previously selected cells
merged_counts_perturb_2guide_object@meta.data$guide <- gsub("\\-.*","", merged_counts_perturb_2guide_object$feature_call)
##extract row names of two subset and combine
rn1 <- rownames(merged_counts_perturb_1guide_object@meta.data)
rn2 <- rownames(merged_counts_perturb_2guide_object@meta.data)
all_rn <- c(rn1, rn2)
write.csv(all_rn, "/home/gzq0002/all_rn")
##only kept all_rn cells with clean guides information
row_ids <- all_rn
merged_counts_perturb1 <- merged_counts_perturb[,(colnames(merged_counts_perturb) %in% row_ids)]
merged_counts_perturb1[[]]
##specify the guide information##
merged_counts_perturb1@meta.data$guide <- gsub("\\-.*","", merged_counts_perturb1$feature_call)
options(repr.plot.height = 4, repr.plot.width = 6)
DimPlot(merged_counts_perturb1, group.by = "group1", raster=FALSE)
DimPlot(merged_counts_perturb1, raster=FALSE)

In [ ]:
table(merged_counts_perturb1$guide)

In [ ]:
#####harmony#####

In [ ]:
##subset tcdd and pbs#####
selected_identname <- "group1"
selected_ident <- c("tcdd","pbs")
selected_rownumbers <- which(merged_counts_perturb1[[]][,selected_identname] %in% selected_ident)
selected_cells <- rownames(merged_counts_perturb1[[]][selected_rownumbers,])
merged_counts_perturb1_tcdd_pbs <- subset(merged_counts_perturb1, cells = selected_cells) # subset from previously selected cells
FeaturePlot(merged_counts_perturb1_tcdd_pbs, features = "nCount_RNA", min.cutoff = 1000)
###harmony
merged_counts_perturb1_tcdd_pbs <- merged_counts_perturb1_tcdd_pbs %>% 
    RunHarmony("group1", plot_convergence = TRUE)
merged_counts_perturb1_tcdd_pbs[[]]
harmony_embeddings <- Embeddings(merged_counts_perturb1_tcdd_pbs, 'harmony')
harmony_embeddings[1:5, 1:5]
options(repr.plot.height = 5, repr.plot.width = 12)
p1 <- DimPlot(object = merged_counts_perturb1_tcdd_pbs, reduction = "harmony", pt.size = .1, group.by = "group1")
p2 <- VlnPlot(object = merged_counts_perturb1_tcdd_pbs, features = "harmony_1", group.by = "group1", pt.size = .1)
plot_grid(p1,p2)
merged_counts_perturb1_tcdd_pbs <- merged_counts_perturb1_tcdd_pbs %>% 
    RunUMAP(reduction = "harmony", dims = 1:20) %>% 
    FindNeighbors(reduction = "harmony", dims = 1:20) %>% 
    FindClusters(resolution = 0.5) %>% 
    identity()

In [ ]:
# save seurat object
saveRDS(merged_counts_perturb1_tcdd_pbs, file = "/data/tracy_data/perturbseq_qc_tcddafterharmony.rds")

In [ ]:
##subset cal and dmem#####
selected_identname <- "group1"
selected_ident <- c("cal","dmem")
selected_rownumbers <- which(merged_counts_perturb1[[]][,selected_identname] %in% selected_ident)
selected_cells <- rownames(merged_counts_perturb1[[]][selected_rownumbers,])
merged_counts_perturb1_cal_dmem <- subset(merged_counts_perturb1, cells = selected_cells) # subset from previously selected cells
FeaturePlot(merged_counts_perturb1_cal_dmem, features = "nCount_RNA", min.cutoff = 1000)
###harmony
merged_counts_perturb1_cal_dmem <- merged_counts_perturb1_cal_dmem %>% 
    RunHarmony("group1", plot_convergence = TRUE)
merged_counts_perturb1_cal_dmem[[]]
harmony_embeddings <- Embeddings(merged_counts_perturb1_cal_dmem, 'harmony')
harmony_embeddings[1:5, 1:5]
options(repr.plot.height = 5, repr.plot.width = 12)
p1 <- DimPlot(object = merged_counts_perturb1_cal_dmem, reduction = "harmony", pt.size = .1, group.by = "group1")
p2 <- VlnPlot(object = merged_counts_perturb1_cal_dmem, features = "harmony_1", group.by = "group1", pt.size = .1)
plot_grid(p1,p2)
merged_counts_perturb1_cal_dmem <- merged_counts_perturb1_cal_dmem %>% 
    RunUMAP(reduction = "harmony", dims = 1:20) %>% 
    FindNeighbors(reduction = "harmony", dims = 1:20) %>% 
    FindClusters(resolution = 0.5) %>% 
    identity()

In [ ]:
# save seurat object
saveRDS(merged_counts_perturb1_cal_dmem, file = "/data/tracy_data/perturbseq_qc_calcificationafterharmony.rds")

In [ ]:
########################################################################################################################

In [ ]:
### downstream analysis_add module score_ tcdd and pbs ###

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 10)
DimPlot(merged_counts_perturb1_tcdd_pbs, reduction = "umap", group.by = "group1", pt.size = .1, split.by = 'group1')

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 12)
FeaturePlot(merged_counts_perturb1_tcdd_pbs, features = c("AHR",
"CYP1B1",
"TIPARP"
), ncol = 3)

In [ ]:
table(merged_counts_perturb1_tcdd_pbs$group1)

In [ ]:
merged_counts_perturb1_tcdd_pbs[[]]

In [ ]:
###Addmodule score
##Calculate modulation scores for feature expression programs in single cells
cd_features <- list(c("BMP2", "ALPL","NAB2","PEX7","MGP","SMPD3","SOX9","RUNX2","COL2A1","ACAN","COL1A1","MEF2C","MMP2"))
merged_counts_perturb1_tcdd_pbs <- AddModuleScore(
  object = merged_counts_perturb1_tcdd_pbs,
  features = cd_features,
  ctrl = 5,
  name = 'modulation_score',
    seed = 1
)
head(x = merged_counts_perturb1_tcdd_pbs[])
my_palette <- c("white", "blue", "red")
names(my_palette) <- c("0", "1", "2")
options(repr.plot.height = 5, repr.plot.width = 6)
Idents(merged_counts_perturb1_tcdd_pbs) <- 'modulation_score1'
FeaturePlot(merged_counts_perturb1_tcdd_pbs, features = "modulation_score1", cols = my_palette)
Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('tcdd'))->tcdd
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(tcdd@meta.data, aes(x = guide, y = modulation_score1)) +
  geom_boxplot(outlier.shape = NA)

Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('pbs'))->pbs
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(pbs@meta.data, aes(x = guide, y = modulation_score1)) +
  geom_boxplot(outlier.shape = NA)

In [ ]:
##Calculate ahr scores for feature expression programs in single cells
cd_features <- list(c('AHR', 'P2RX1', 'CLDN14', 'CYP1A1', 'C3orf80', 'EFCAB12', 'RGCC', 'RASL11B', 'IL11', 'IGF1', 'COL7A1', 'NFATC2', 'RIN1', 'GPR68', 'PTGS2', 'AC020571.3', 'AP001007.1', 'SLAMF9', 'PENK', 'AC018755.1'))
merged_counts_perturb1_tcdd_pbs <- AddModuleScore(
  object = merged_counts_perturb1_tcdd_pbs,
  features = cd_features,
  ctrl = 5,
  name = 'ahr_score',
    seed = 1
)

head(x = merged_counts_perturb1_tcdd_pbs[])
my_palette <- c("white", "blue", "red")
names(my_palette) <- c("0", "1", "2")
options(repr.plot.height = 5, repr.plot.width = 6)

Idents(merged_counts_perturb1_tcdd_pbs) <- 'ahr_score1'
FeaturePlot(merged_counts_perturb1_tcdd_pbs, features = "ahr_score1", cols = my_palette)
Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('tcdd'))->tcdd
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(tcdd@meta.data, aes(x = guide, y = ahr_score1)) +
  geom_boxplot(outlier.shape = NA)

Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('pbs'))->pbs
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(pbs@meta.data, aes(x = guide, y = ahr_score1)) +
  geom_boxplot(outlier.shape = NA)

In [ ]:
##Calculate TCDD scores for feature expression programs in single cells
cd_features <- list(c('NPTX1', 'ATP1B2', 'CYP1A1', 'CD7', 'HS3ST2', 'FGF18', 'PDK4', 'F2RL3', 'NKAIN1', 'ERBB4', 'IL1RN', 'MIR124-1','OLFM2', 'MIR4635', 'NKD2','CYP1B1', 'SLC16A6', 'SECTM1', 'FENDRR', 'ABCG2'))
merged_counts_perturb1_tcdd_pbs <- AddModuleScore(
  object = merged_counts_perturb1_tcdd_pbs,
  features = cd_features,
  ctrl = 5,
  name = 'tcdd_score',
    seed = 1
)
head(x = merged_counts_perturb1_tcdd_pbs[])
my_palette <- c("white", "blue", "red")
names(my_palette) <- c("0", "1", "2")
options(repr.plot.height = 5, repr.plot.width = 6)

Idents(merged_counts_perturb1_tcdd_pbs) <- 'tcdd_score1'
FeaturePlot(merged_counts_perturb1_tcdd_pbs, features = "tcdd_score1", cols = my_palette)
Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('tcdd'))->tcdd
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(tcdd@meta.data, aes(x = guide, y = tcdd_score1)) +
  geom_boxplot(outlier.shape = NA)

Idents(merged_counts_perturb1_tcdd_pbs)<-merged_counts_perturb1_tcdd_pbs$group1
subset(merged_counts_perturb1_tcdd_pbs,idents = c('pbs'))->pbs
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(pbs@meta.data, aes(x = guide, y = tcdd_score1)) +
  geom_boxplot(outlier.shape = NA)

In [ ]:
###calculate p values for tcdd cells
library(dplyr)
meta <- tcdd@meta.data

In [ ]:
# 设置列名
score_col <- "modulation_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL"   # 改成你的control名称

# 提取control
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)

# 获取所有其他guide
guides <- setdiff(unique(meta[[guide_col]]), control_label)

# 结果表
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)

# 循环做Wilcoxon
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}

# BH校正
res$FDR <- p.adjust(res$p_value, method = "BH")

# 排序
res <- res %>% arrange(p_value)
print(res)

In [ ]:
score_col <- "ahr_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
score_col <- "tcdd_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
###calculate p values for pbs cells
library(dplyr)
meta <- pbs@meta.data

In [ ]:
score_col <- "modulation_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
score_col <- "ahr_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
score_col <- "tcdd_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
### downstream analysis_add module score_cal and dmem ###

In [ ]:
##Calculate calcification scores for feature expression programs in single cells
cd_features <- list(c('RP11-54O7.3', 'SAMD11', 'RASL11B', 'RP4-625H18.2', 'ANO1', 'COMP', 'RP1-167F1.2', 'SERPIND1', 'ID3', 'ECSCR', 'MMP1', 'AC133785.1', 'ID1', 'NOTUM', 'FXYD6', 'RP11-49I11.2', 'RELN', 'PDE3B', 'ID4', 'TINAGL1'))
merged_counts_perturb1_cal_dmem <- AddModuleScore(
  object = merged_counts_perturb1_cal_dmem,
  features = cd_features,
  ctrl = 5,
  name = 'calcification_score',
    seed = 1
)

head(x = merged_counts_perturb1_cal_dmem[])
my_palette <- c("white", "blue", "red")
names(my_palette) <- c("0", "1", "2")
options(repr.plot.height = 5, repr.plot.width = 6)

Idents(merged_counts_perturb1_cal_dmem) <- 'calcification_score1'
FeaturePlot(merged_counts_perturb1_cal_dmem, features = "calcification_score1", cols = my_palette)
Idents(merged_counts_perturb1_cal_dmem)<-merged_counts_perturb1_cal_dmem$group1
subset(merged_counts_perturb1_cal_dmem,idents = c('cal'))->calcification
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(calcification@meta.data, aes(x = guide, y = calcification_score1)) +
  geom_boxplot(outlier.shape = NA)

Idents(merged_counts_perturb1_cal_dmem)<-merged_counts_perturb1_cal_dmem$group1
subset(merged_counts_perturb1_cal_dmem,idents = c('dmem'))->dmem
options(repr.plot.height = 6, repr.plot.width = 8)
ggplot(dmem@meta.data, aes(x = guide, y = calcification_score1)) +
  geom_boxplot(outlier.shape = NA)

In [ ]:
###calcification p value
library(dplyr)
meta <- calcification@meta.data

score_col <- "calcification_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
###dmem p value
library(dplyr)
meta <- dmem@meta.data

score_col <- "calcification_score1"
guide_col <- "guide"
control_label <- "NEG_CTRL" 
meta_ctrl <- meta %>% filter(!!sym(guide_col) == control_label)
guides <- setdiff(unique(meta[[guide_col]]), control_label)
res <- data.frame(guide = guides, p_value = NA, median_diff = NA)
for (g in guides) {
  meta_g <- meta %>% filter(!!sym(guide_col) == g)
  test <- wilcox.test(meta_g[[score_col]], meta_ctrl[[score_col]], exact = FALSE)
  res[res$guide == g, "p_value"] <- test$p.value
  res[res$guide == g, "median_diff"] <- median(meta_g[[score_col]], na.rm = TRUE) - median(meta_ctrl[[score_col]], na.rm = TRUE)
}
res$FDR <- p.adjust(res$p_value, method = "BH")
res <- res %>% arrange(p_value)
print(res)

In [ ]:
############################  mixscale  ##################################

In [ ]:
library(Seurat)
library(ggridges)
library(ggplot2)
library(Mixscale)
library(EnhancedVolcano)

In [ ]:
merged_counts_perturb1[[]]

In [ ]:
# Read in the expression matrix The first row is a header row, the first column is rownames
exp.mat <- read.table(file = "/VOSNE/tracy/nestorawa_forcellcycle_expressionMatrix.txt", header = TRUE,
    as.is = TRUE, row.names = 1)
# A list of cell cycle markers, from Tirosh et al, 2015, is loaded with Seurat.  We can segregate this list into markers of G2/M phase and markers of S phase
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes
# Assign cell cycle store
merged_counts_perturb_cell_cycle <- CellCycleScoring(merged_counts_perturb1, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
# Running a PCA on cell cycle genes reveals, unsurprisingly, that cells separate entirely by phase
merged_counts_perturb_cell_cycle <- RunPCA(merged_counts_perturb_cell_cycle, features = c(s.genes, g2m.genes))
DimPlot(merged_counts_perturb_cell_cycle)

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
p1=DimPlot(merged_counts_perturb1, group.by = "group1", raster=FALSE)
p2=DimPlot(merged_counts_perturb1, raster=FALSE)
p3=DimPlot(merged_counts_perturb_cell_cycle)
p1+p2+p3

In [ ]:
table(merged_counts_perturb_cell_cycle$group1)

In [ ]:
table(merged_counts_perturb_cell_cycle$guide)

In [ ]:
subset(merged_counts_perturb_cell_cycle,subset=group1=="pbs")->merged_counts_perturb_cell_cycle

In [ ]:
table(merged_counts_perturb_cell_cycle$group1)

In [ ]:
merged_counts_perturb_cell_cycle[[]]

In [ ]:
names(merged_counts_perturb_cell_cycle@meta.data)[15] <- 'gene'

In [ ]:
merged_counts_perturb_cell_cycle@meta.data

In [ ]:
seurat_obj <- CalcPerturbSig(
    object = merged_counts_perturb_cell_cycle, 
    assay = "RNA", 
    slot = "data", 
    gd.class ="gene", 
    nt.cell.class = "NEG_CTRL", 
    reduction = "pca", 
    ndims = 40, 
    num.neighbors = 20, 
    new.assay.name = "PRTB", 
    split.by = NULL) 

In [ ]:
# Mixscale
seurat_obj = RunMixscale(
    object = seurat_obj, 
    assay = "PRTB", 
    slot = "scale.data", 
    labels = "gene", 
    nt.class.name = "NEG_CTRL", 
    min.de.genes = 5, 
    logfc.threshold = 0.2,
    de.assay = "RNA",
    max.de.genes = 100, 
    new.class.name = "mixscale_score", 
    fine.mode = F, 
    verbose = F, 
    split.by = NULL)

In [ ]:
# a. Check the distribution of the scores for the perturbations
RidgePlot(
    seurat_obj,
    features = "mixscale_score",
    group.by = "gene") + NoLegend()

In [ ]:
# b. Check if the scores correlate with the expression level of the target gene itself
Mixscale_ScatterPlot(object = seurat_obj, 
                     nt.class.name = "NEG_CTRL", 
                     slct.ident = unique(seurat_obj$gene)[unique(seurat_obj$gene) != "NEG_CTRL"][1:10], 
                     nbin = 10, 
                     facet_wrap = "gene") + NoLegend()

In [ ]:
table(seurat_obj$gene)

In [ ]:
# run score-based weighted DE test for selected perturbations. It will return a list of data frames (one for each perturbation)
de_res = Run_wmvRegDE(object = seurat_obj, assay = "RNA", slot = "counts",
                      labels = "gene", nt.class.name = "NEG_CTRL", 
                      PRTB_list = c("ARNT", "NFE2L2", "SMAD3","HSP90AA1","KLF4","SMARCA4","SOX9","TCF21","TIPARP","TWIST1"),
                      logfc.threshold = 0.2, 
                      split.by = NULL)

# have a quick look at the DE results
head(de_res[[1]])

In [ ]:
de_res

In [ ]:
str(de_res)

In [ ]:
# Create a directory to save CSV files if it doesn't exist
if (!file.exists("output_csv")) {
  dir.create("output_csv")
}
# Iterate over each data frame in the list
for (i in seq_along(de_res)) {
  # Generate a unique file name for each data frame
  file_name <- paste0("output_csv/data_frame_", i, ".csv")
  
  # Save the data frame to CSV
  write.csv(de_res[[i]], file = file_name, row.names = FALSE)
}

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["ARNT"]][order(de_res[["ARNT"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "ARNT", 
                   mixscale.score.name = "mixscale_score",
                   features = c("ARNT", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["HSP90AA1"]][order(de_res[["HSP90AA1"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "HSP90AA1", 
                   mixscale.score.name = "mixscale_score",
                   features = c("HSP90AA1", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["KLF4"]][order(de_res[["KLF4"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "KLF4", 
                   mixscale.score.name = "mixscale_score",
                   features = c("KLF4", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["NFE2L2"]][order(de_res[["NFE2L2"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "NFE2L2", 
                   mixscale.score.name = "mixscale_score",
                   features = c("NFE2L2", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["SMAD3"]][order(de_res[["SMAD3"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "SMAD3", 
                   mixscale.score.name = "mixscale_score",
                   features = c("SMAD3", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["SMARCA4"]][order(de_res[["SMARCA4"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "SMARCA4", 
                   mixscale.score.name = "mixscale_score",
                   features = c("SMARCA4", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["SOX9"]][order(de_res[["SOX9"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "SOX9", 
                   mixscale.score.name = "mixscale_score",
                   features = c("SOX9", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["TCF21"]][order(de_res[["TCF21"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "TCF21", 
                   mixscale.score.name = "mixscale_score",
                   features = c("TCF21", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["TIPARP"]][order(de_res[["TIPARP"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "TIPARP", 
                   mixscale.score.name = "mixscale_score",
                   features = c("TIPARP", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
# select the top 20 DE genes from the perturbation
top_res = de_res[["TWIST1"]][order(de_res[["TWIST1"]]$p_weight)[1:20], ]
# order the DE genes based on its log-fold-change
top_DEG = rownames(top_res[order(top_res$log2FC, decreasing = T), ])

# heatmap for the top DE genes. cells ordered by Mixscale scores. The expression level of the target gene will be plotted in the first row.
Mixscale_DoHeatmap(object = seurat_obj, 
                   labels = "gene", 
                   nt.class.name = "NEG_CTRL", 
                   slct.ident = "TWIST1", 
                   mixscale.score.name = "mixscale_score",
                   features = c("TWIST1", top_DEG), angle = 0, hjust = 0.5) 

In [ ]:
###volcano plots
library(EnhancedVolcano)

In [ ]:
# ARNT
read.csv(file = "/home/gzq0002/arnt.csv", header = TRUE) ->arnt
options(repr.plot.height = 10, repr.plot.width = 10)
EnhancedVolcano(arnt , 
                arnt$gene_ID,
                x ="log2FC", 
                y ="p_weight",
               lab = as.character(arnt$gene_ID, pointSize = 4.0))

In [ ]:
# HSP90AA1
read.csv(file = "/home/gzq0002/hsp90aa1.csv", header = TRUE) ->hsp90aa1
options(repr.plot.height = 10, repr.plot.width = 10)
EnhancedVolcano(hsp90aa1 , 
                hsp90aa1$gene_ID,
                x ="log2FC", 
                y ="p_weight",
                ylim = c(0, 15),
               lab = as.character(hsp90aa1$gene_ID, pointSize = 4.0))

In [ ]:
# HSP90AA1
read.csv(file = "/home/gzq0002/hsp90aa1.csv", header = TRUE) ->hsp90aa1
options(repr.plot.height = 10, repr.plot.width = 10)
EnhancedVolcano(hsp90aa1 , 
                hsp90aa1$gene_ID,
                x ="log2FC", 
                y ="p_weight",
               lab = as.character(hsp90aa1$gene_ID, pointSize = 4.0))

In [ ]:
# tcf21
read.csv(file = "/home/gzq0002/tcf21.csv", header = TRUE) ->tcf21
options(repr.plot.height = 10, repr.plot.width = 10)
EnhancedVolcano(tcf21 , 
                tcf21$gene_ID,
                x ="log2FC", 
                y ="p_weight",
               lab = as.character(tcf21$gene_ID, pointSize = 4.0))

In [ ]:
# smarca4
read.csv(file = "/home/gzq0002/smarca4.csv", header = TRUE) ->smarca4
options(repr.plot.height = 10, repr.plot.width = 10)
EnhancedVolcano(smarca4 , 
                smarca4$gene_ID,
                x ="log2FC", 
                y ="p_weight",
               lab = as.character(smarca4$gene_ID, pointSize = 4.0))